In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
##%reload_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-72a1u9l4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-72a1u9l4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=637d639a499674e97e5d36958a5fb5de85ba33143fd2b23a8c0da0e66f5cae37
  Stored in directory: /tmp/pip-ephem-wheel-cache-smj_rx19/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cu

#include <stdio.h>

__global__ void cuda_hello()
{
    printf("hello world from gpu\n");
}

int main()
{
    cuda_hello<<<1,5>>>();
    cudaDeviceSynchronize(); // sync required in colab enviroment

    return 0;
}

hello world from gpu
hello world from gpu
hello world from gpu
hello world from gpu
hello world from gpu



Simple Memcpy Scenario

In [4]:
%%cu

#include <stdio.h>

__global__ void addKernel(int *c , const int *a, const int *b)
{
    int i = threadIdx.x;
    c[i] = a[i] + b[i];
}

int main()
{
    const int SIZE = 5;
    const int a[SIZE] = {1,2,3,4,5};
    const int b[SIZE] = {10,20,30,40,50};
 
    int c[SIZE] = {0};
 
    int *dev_a = 0;
    int *dev_b = 0;
    int *dev_c = 0;
 
    cudaMalloc((void**)&dev_a, SIZE * sizeof(int));
    cudaMalloc((void**)&dev_b, SIZE * sizeof(int));
    cudaMalloc((void**)&dev_c, SIZE * sizeof(int));
 
    cudaMemcpy(dev_a, a, SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_c, c, SIZE * sizeof(int), cudaMemcpyHostToDevice);
 
    addKernel<<<1,SIZE>>>(dev_c, dev_a, dev_b);
    cudaDeviceSynchronize(); // sync required in colab enviroment
 
    cudaMemcpy(c, dev_c, SIZE * sizeof(int), cudaMemcpyDeviceToHost);
 
    printf("{1,2,3,4,5} + {10,20,30,40,50} = {%d, %d, %d, %d, %d}\n",c[0],c[1],c[2], c[3],c[4]);
 
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

{1,2,3,4,5} + {10,20,30,40,50} = {11, 22, 33, 44, 55}



Kernel with 2D Indexing

In [5]:
%%cu

#include <stdio.h>

__global__ void kernel(int *a, int dimx, int dimy)
{
    int ix = blockIdx.x * blockDim.x + threadIdx.x;
    int iy = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = iy * dimx + ix;

    a[idx] = idx + 1;
}


int main()
{
    int dimx = 8;
    int dimy = 8;

    int num_bytes = dimx*dimy*sizeof(int);

    int *dev_a;
    int *a; 

    a = (int*)malloc(num_bytes);
    cudaMalloc((void**)&dev_a,num_bytes);

    cudaMemset(dev_a, 0, num_bytes);

    dim3 DimGrid(4,4); // # of blocks in grid = 4 x 4 = 16
    dim3 DimBlock(2,2); // # of threads in a blcok = 2 x 2 = 4

    kernel<<<DimGrid,DimBlock>>>(dev_a, dimx, dimy);

    cudaMemcpy(a, dev_a, num_bytes, cudaMemcpyDeviceToHost);
    
    for(int row=0; row<dimy; row++)
    {
        for (int col=0; col<dimx; col++)
        {
            printf("(%d,%d)  %d\t", col, row, a[row*dimx+col]);
        }
        printf("\n");
    }
    free(a);
    cudaFree(dev_a);

    return 0;
}


(0,0)  1	(1,0)  2	(2,0)  3	(3,0)  4	(4,0)  5	(5,0)  6	(6,0)  7	(7,0)  8	
(0,1)  9	(1,1)  10	(2,1)  11	(3,1)  12	(4,1)  13	(5,1)  14	(6,1)  15	(7,1)  16	
(0,2)  17	(1,2)  18	(2,2)  19	(3,2)  20	(4,2)  21	(5,2)  22	(6,2)  23	(7,2)  24	
(0,3)  25	(1,3)  26	(2,3)  27	(3,3)  28	(4,3)  29	(5,3)  30	(6,3)  31	(7,3)  32	
(0,4)  33	(1,4)  34	(2,4)  35	(3,4)  36	(4,4)  37	(5,4)  38	(6,4)  39	(7,4)  40	
(0,5)  41	(1,5)  42	(2,5)  43	(3,5)  44	(4,5)  45	(5,5)  46	(6,5)  47	(7,5)  48	
(0,6)  49	(1,6)  50	(2,6)  51	(3,6)  52	(4,6)  53	(5,6)  54	(6,6)  55	(7,6)  56	
(0,7)  57	(1,7)  58	(2,7)  59	(3,7)  60	(4,7)  61	(5,7)  62	(6,7)  63	(7,7)  64	



Matrix Addition in CUDA

In [6]:
%%cu

#include <stdio.h>
__global__ void addKernel(int* c, const int* a, const int* b)
{
    int x = threadIdx.x;
    int y = threadIdx.y;
    int i = y * (blockDim.x) + x; // index = y * WIDTH + x
    c[i] = a[i] + b[i];
}

int main(){
    const int WIDTH=5;
    int a[WIDTH][WIDTH];
    int b[WIDTH][WIDTH];
    int c[WIDTH][WIDTH] = { 0 };

    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            a[y][x] = y*10+x;
            b[y][x] = (y*10+x)*100;
        }
    }
    
    int *dev_a, *dev_b, *dev_c = 0; // GPU does not know the array structure of dev_a, dev_b, dev_c
    cudaMalloc((void**)&dev_a, WIDTH*WIDTH*sizeof(int)); // Memory allocation (WIDTH*WIDTH)
    cudaMalloc((void**)&dev_b, WIDTH*WIDTH*sizeof(int)); // Memory allocation (WIDTH*WIDTH)
    cudaMalloc((void**)&dev_c, WIDTH*WIDTH*sizeof(int)); // Memory allocation (WIDTH*WIDTH)

    cudaMemcpy(dev_a, a, WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);

    dim3 DimBlock(WIDTH,WIDTH);
    addKernel <<<1,DimBlock>>>(dev_c,dev_a,dev_b);

    cudaMemcpy(c, dev_c, WIDTH*WIDTH*sizeof(int), cudaMemcpyDeviceToHost);
    
    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            printf("%5d ", c[y][x]);
        }
        printf("\n");
    }

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


    0   101   202   303   404 
 1010  1111  1212  1313  1414 
 2020  2121  2222  2323  2424 
 3030  3131  3232  3333  3434 
 4040  4141  4242  4343  4444 



Matrix Multiplication in Host (CPU)

In [7]:
%%cu

#include <stdio.h>

int main()
{
    const int WIDTH=5;
    int a[WIDTH][WIDTH];
    int b[WIDTH][WIDTH];
    int c[WIDTH][WIDTH] = { 0 };

    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            a[y][x] = y+x;
            b[y][x] = y+x;
        }
    }
    
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            int sum=0;
            for (int k=0; k<WIDTH; k++)
            {
                sum += a[y][k] * b[k][x];
            }
            c[y][x] = sum;
        }
    }

    printf("--------Matrix A---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", a[y][x]);
        }
        printf("\n");
    }
    
    printf("--------Matrix B---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", b[y][x]);
        }
        printf("\n");
    }

    printf("--------Matrix C---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", c[y][x]);
        }
        printf("\n");
    }

    return 0;
}

--------Matrix A---------
    0     1     2     3     4 
    1     2     3     4     5 
    2     3     4     5     6 
    3     4     5     6     7 
    4     5     6     7     8 
--------Matrix B---------
    0     1     2     3     4 
    1     2     3     4     5 
    2     3     4     5     6 
    3     4     5     6     7 
    4     5     6     7     8 
--------Matrix C---------
   30    40    50    60    70 
   40    55    70    85   100 
   50    70    90   110   130 
   60    85   110   135   160 
   70   100   130   160   190 



Matrix Multiplication in Host (GPU)

In [8]:
%%cu

#include <stdio.h>

__global__ void multiply(int* c, const int* a, const int* b, const int WIDTH)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int i = y * WIDTH + x;
    
    int sum=0;
    for (int k=0; k<WIDTH; k++)
    {
      sum += a[y*WIDTH+k] * b[k*WIDTH+x];
    }
    c[i] = sum;
}


int main()
{
    const int WIDTH=5;
    int a[WIDTH][WIDTH];
    int b[WIDTH][WIDTH];
    int c[WIDTH][WIDTH] = { 0 };

    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            a[y][x] = y+x;
            b[y][x] = y+x;
        }
    }
    
    int *dev_a, *dev_b, *dev_c = 0;
    cudaMalloc((void**)&dev_a,WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_b,WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_c,WIDTH*WIDTH*sizeof(int));
 
    cudaMemcpy(dev_a, a ,WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b ,WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
 
    dim3 DimBlock(5,5);
 
    multiply<<<1,DimBlock>>>(dev_c, dev_a, dev_b, WIDTH);
 
    cudaMemcpy(c, dev_c,WIDTH*WIDTH*sizeof(int), cudaMemcpyDeviceToHost);

    printf("--------Matrix A---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", a[y][x]);
        }
        printf("\n");
    }
    
    printf("--------Matrix B---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", b[y][x]);
        }
        printf("\n");
    }

    printf("--------Matrix C---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", c[y][x]);
        }
        printf("\n");
    }
 
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}

--------Matrix A---------
    0     1     2     3     4 
    1     2     3     4     5 
    2     3     4     5     6 
    3     4     5     6     7 
    4     5     6     7     8 
--------Matrix B---------
    0     1     2     3     4 
    1     2     3     4     5 
    2     3     4     5     6 
    3     4     5     6     7 
    4     5     6     7     8 
--------Matrix C---------
   30    40    50    60    70 
   40    55    70    85   100 
   50    70    90   110   130 
   60    85   110   135   160 
   70   100   130   160   190 



Kernel Launch

In [9]:
%%cu

#include <stdio.h>

__global__ void multiply(int* c, const int* a, const int* b, const int WIDTH)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int i = y * WIDTH + x;
    
    int sum=0;
    for (int k=0; k<WIDTH; k++)
    {
      sum += a[y*WIDTH+k] * b[k*WIDTH+x];
    }
    c[i] = sum;
}


int main()
{
    const int WIDTH = 8;
    const int TILE_WIDTH = 2;
    int a[WIDTH][WIDTH];
    int b[WIDTH][WIDTH];
    int c[WIDTH][WIDTH] = { 0 };

    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            a[y][x] = y+x;
            b[y][x] = y+x;
        }
    }
    
    int *dev_a, *dev_b, *dev_c = 0;
    cudaMalloc((void**)&dev_a,WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_b,WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_c,WIDTH*WIDTH*sizeof(int));
 
    cudaMemcpy(dev_a, a ,WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b ,WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
 
    dim3 dimGrid(WIDTH/TILE_WIDTH,WIDTH/TILE_WIDTH,1);
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);
 
    multiply<<<dimGrid,dimBlock>>>(dev_c, dev_a, dev_b, WIDTH);
 
    cudaMemcpy(c, dev_c,WIDTH*WIDTH*sizeof(int), cudaMemcpyDeviceToHost);

    printf("--------Matrix A---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", a[y][x]);
        }
        printf("\n");
    }
    
    printf("--------Matrix B---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", b[y][x]);
        }
        printf("\n");
    }

    printf("--------Matrix C---------\n");
    for (int y=0; y<WIDTH; y++)
    {
        for (int x=0; x<WIDTH; x++)
        {
            printf("%5d ", c[y][x]);
        }
        printf("\n");
    }
 
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}

--------Matrix A---------
    0     1     2     3     4     5     6     7 
    1     2     3     4     5     6     7     8 
    2     3     4     5     6     7     8     9 
    3     4     5     6     7     8     9    10 
    4     5     6     7     8     9    10    11 
    5     6     7     8     9    10    11    12 
    6     7     8     9    10    11    12    13 
    7     8     9    10    11    12    13    14 
--------Matrix B---------
    0     1     2     3     4     5     6     7 
    1     2     3     4     5     6     7     8 
    2     3     4     5     6     7     8     9 
    3     4     5     6     7     8     9    10 
    4     5     6     7     8     9    10    11 
    5     6     7     8     9    10    11    12 
    6     7     8     9    10    11    12    13 
    7     8     9    10    11    12    13    14 
--------Matrix C---------
  140   168   196   224   252   280   308   336 
  168   204   240   276   312   348   384   420 
  196   240   284   328   372   416   46

Host version

In [10]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <chrono>

#define GRIDSIZE (8*1024) // 8K
#define BLOCKSIZE 1024 // 1K
#define TOTALSIZE (GRIDSIZE * BLOCKSIZE) // 32MB (8K * 1K * 4B)


void genData(float* ptr, unsigned int size){
    while (size--){
        *ptr++ = (float)(rand() % 1000)/1000.0F;
    }
}

void adjDiff(float* dst, const float* src, unsigned int size)
{
    for(int i = 1; i<size; ++i)
    {
        dst[i] = src[i] - src[i-1];
    }
}

int main()
{
    float* pSource = NULL;
    float* pResult = NULL;

    float* pSourceDev = NULL;
    float* pResultDev = NULL;

    int i;

    pSource = (float*)malloc(TOTALSIZE * sizeof(float));
    pResult = (float*)malloc(TOTALSIZE * sizeof(float));

    genData(pSource, TOTALSIZE);
 
    //start timer
    std::chrono::system_clock::time_point start = std::chrono::system_clock::now();
    // adjacent difference
    adjDiff(pResult, pSource, TOTALSIZE);
    //end timer
    std::chrono::system_clock::time_point end = std::chrono::system_clock::now();
    std::chrono::nanoseconds duration_nano = end - start;
    cudaMemcpy(pResult, pResultDev, TOTALSIZE * sizeof(float), cudaMemcpyHostToDevice);
    printf("Elapsed Time: %lld ns\n", duration_nano);
 
    //print sample cases 
    i = 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE - 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE/2;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
 
    free(pSource);
    free(pResult);
    return 0;
}

Elapsed Time: 42622234 ns
i=      1: 0.503000=0.886000-0.383000
i=8388607: 0.700000=0.820000-0.120000
i=4194304: 0.609000=0.917000-0.308000



Device version

In [11]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <chrono>

#define GRIDSIZE (8*1024) // 8K
#define BLOCKSIZE 1024 // 1K
#define TOTALSIZE (GRIDSIZE * BLOCKSIZE) // 32MB (8K * 1K * 4B)

void genData(float* ptr, unsigned int size){
    while (size--){
        *ptr++ = (float)(rand() % 1000)/1000.0F;
    }
}

__global__ void adjDiff(float* result, float* input){
    unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i>0){
        float x_i = input[i];
        float x_i_m1 = input[i-1];
        result[i] = x_i - x_i_m1;
    }
}

int main()
{
    float* pSource = NULL;
    float* pResult = NULL;

    float* pSourceDev = NULL;
    float* pResultDev = NULL;

    int i;

    pSource = (float*)malloc(TOTALSIZE * sizeof(float));
    pResult = (float*)malloc(TOTALSIZE * sizeof(float));

    genData(pSource, TOTALSIZE);

    pResult[0] = 0.0F;

    cudaMalloc((void**)&pSourceDev, TOTALSIZE * sizeof(float));
    cudaMalloc((void**)&pResultDev, TOTALSIZE * sizeof(float));
    cudaMemcpy(pSourceDev, pSource, TOTALSIZE * sizeof(float), cudaMemcpyHostToDevice);
    
    //start timer
    std::chrono::system_clock::time_point start = std::chrono::system_clock::now();
    // adjacent difference
   
    dim3 DimGrid(GRIDSIZE, 1, 1);
    dim3 DimBlock(BLOCKSIZE, 1, 1);

    adjDiff<<<DimGrid,DimBlock>>>(pResultDev,pSourceDev);

    //end timer
    std::chrono::system_clock::time_point end = std::chrono::system_clock::now();
    std::chrono::nanoseconds duration_nano = end - start;
    cudaMemcpy(pResult, pResultDev, TOTALSIZE * sizeof(float), cudaMemcpyHostToDevice);
    printf("Elapsed Time: %lld ns\n", duration_nano);


    //print sample cases 
    i = 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE - 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE/2;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);

    free(pSource);
    free(pResult);
    cudaFree(pSourceDev);
    cudaFree(pResultDev);

    return 0;
}


Elapsed Time: 22754 ns
i=      1: 0.000000=0.886000-0.383000
i=8388607: 0.000000=0.820000-0.120000
i=4194304: 0.000000=0.917000-0.308000



In [12]:
%cd /usr/local/cuda-11.0/samples/1_Utilities/deviceQuery/
!make
!ls
!./deviceQuery

/usr/local/cuda-11.0/samples/1_Utilities/deviceQuery
/usr/local/cuda-11.0/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_80,code=compute_80 -o deviceQuery.o -c deviceQuery.cpp
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
/usr/local/cuda-11.0/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=

AdjDiff Shared Version

In [13]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <chrono>

#define GRIDSIZE (8*1024) // 8K
#define BLOCKSIZE 1024 // 1K
#define TOTALSIZE (GRIDSIZE * BLOCKSIZE) // 32MB (8K * 1K * 4B)

void genData(float* ptr, unsigned int size){
    while (size--){
        *ptr++ = (float)(rand() % 1000)/1000.0F;
    }
}

__global__ void adjDiff(float* result, float* input){
    __shared__ float s_data[BLOCKSIZE];
    unsigned int tx = threadIdx.x;
    unsigned int i = blockIdx.x * blockDim.x + tx;
    s_data[tx] = input[i]; // global --> shared data movement 
    __syncthreads();
    if (tx>0){
        result[i] = s_data[tx] - s_data[tx-1];
    } 
    else if(i>0){
        result[i] = s_data[tx] - input[i-1];
    }
    __syncthreads();
}

int main(){
    float* pSource = NULL;
    float* pResult = NULL;

    float* pSourceDev = NULL;
    float* pResultDev = NULL;

    int i;

    pSource = (float*)malloc(TOTALSIZE * sizeof(float));
    pResult = (float*)malloc(TOTALSIZE * sizeof(float));

    genData(pSource, TOTALSIZE);

    pResult[0] = 0.0F;

    

    cudaMalloc((void**)&pSourceDev, TOTALSIZE * sizeof(float));
    cudaMalloc((void**)&pResultDev, TOTALSIZE * sizeof(float));
    cudaMemcpy(pSourceDev, pSource, TOTALSIZE * sizeof(float), cudaMemcpyHostToDevice);
    
    // adjacent difference
    //adjDiff(pResult, pSource, TOTALSIZE);
    dim3 DimGrid(GRIDSIZE, 1, 1);
    dim3 DimBlock(BLOCKSIZE, 1, 1);

    //start timer
    std::chrono::system_clock::time_point start = std::chrono::system_clock::now();
    adjDiff<<<DimGrid,DimBlock>>>(pResultDev,pSourceDev);

    //end timer
    std::chrono::system_clock::time_point end = std::chrono::system_clock::now();
    std::chrono::nanoseconds duration_nano = end - start;

    cudaMemcpy(pResult, pResultDev, TOTALSIZE * sizeof(float), cudaMemcpyDeviceToHost);

    printf("Elapsed Time: %lld ns\n", duration_nano);


    //print sample cases 
    i = 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE - 1;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);
    i = TOTALSIZE/2;
    printf("i=%7d: %f=%f-%f\n", i, pResult[i], pSource[i], pSource[i-1]);

    free(pSource);
    free(pResult);
    cudaFree(pSourceDev);
    cudaFree(pResultDev);

    return 0;
}


Elapsed Time: 17994 ns
i=      1: 0.503000=0.886000-0.383000
i=8388607: 0.700000=0.820000-0.120000
i=4194304: 0.609000=0.917000-0.308000



In [14]:
%%cu

#include <stdio.h>
#include <chrono>

#define WIDTH 512
#define TILE_WIDTH 32

__global__ void multiply(int* c, const int* a, const int* b){
    __shared__ int ds_A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int ds_B[TILE_WIDTH][TILE_WIDTH];

    int Row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int Col = blockIdx.x * TILE_WIDTH + threadIdx.x;

    int sub_C = 0;

    for (int m=0; m < WIDTH/TILE_WIDTH; ++m){
        ds_A[threadIdx.y][threadIdx.x] = a[Row*WIDTH + m*TILE_WIDTH+threadIdx.x];
        ds_B[threadIdx.y][threadIdx.x] = b[(m*TILE_WIDTH+threadIdx.y)*WIDTH + Col];
        __syncthreads();
        for (int k=0; k<TILE_WIDTH; ++k){
            sub_C += ds_A[threadIdx.y][k] * ds_B[k][threadIdx.x];
        }
        __syncthreads();
    }
    
    c[Row*WIDTH+Col] = sub_C;
}

int main(){
    //const int WIDTH=512;
    //const int TILE_WIDTH=32;
    int a[WIDTH][WIDTH];
    int b[WIDTH][WIDTH];
    int c[WIDTH][WIDTH] = { 0 };

    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            a[y][x] = y;
            b[y][x] = x;
        }
    }
    int num_bytes = WIDTH*WIDTH*sizeof(int);

    int *dev_a, *dev_b, *dev_c = 0;
    cudaMalloc((void**)&dev_c, WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_a, WIDTH*WIDTH*sizeof(int));
    cudaMalloc((void**)&dev_b, WIDTH*WIDTH*sizeof(int));

    cudaMemcpy(dev_a, a, WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, WIDTH*WIDTH*sizeof(int), cudaMemcpyHostToDevice);

    dim3 DimGrid(WIDTH/TILE_WIDTH,WIDTH/TILE_WIDTH);
    dim3 DimBlock(TILE_WIDTH, TILE_WIDTH);

    //start timer
    std::chrono::system_clock::time_point start = std::chrono::system_clock::now();

    multiply<<<DimGrid,DimBlock>>>(dev_c, dev_a, dev_b);

    //end timer
    std::chrono::system_clock::time_point end = std::chrono::system_clock::now();
    std::chrono::nanoseconds duration_nano = end - start;

    printf("Elapsed Time: %lld ns\n", duration_nano);

    cudaMemcpy(c, dev_c, WIDTH*WIDTH*sizeof(int), cudaMemcpyDeviceToHost);

    //for (int y=0; y<WIDTH; y++){
    //    for (int x=0; x<WIDTH; x++){
    //        // Kernel
    //    }
    //}

    /*
    printf("--------Matrix A---------\n");
    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            printf("%5d ", a[y][x]);
        }
        printf("\n");
    }
    
    printf("--------Matrix B---------\n");
    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            printf("%5d ", b[y][x]);
        }
        printf("\n");
    }

    printf("--------Matrix C---------\n");
    for (int y=0; y<WIDTH; y++){
        for (int x=0; x<WIDTH; x++){
            printf("%5d ", c[y][x]);
        }
        printf("\n");
    }
    */
    
    //print the result

    int i=0,j=0;
    printf("c[%4d][%4d] = %d\n",i,j,c[i][j]);
    i=WIDTH/2;
    j=WIDTH/2;
    printf("c[%4d][%4d] = %d\n",i,j,c[i][j]);
    i=WIDTH-1;
    j=WIDTH-1;
    printf("c[%4d][%4d] = %d\n",i,j,c[i][j]);

    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);

    return 0;
}

Elapsed Time: 15596 ns
c[   0][   0] = 0
c[ 256][ 256] = 33554432
c[ 511][ 511] = 133693952

